<a href="https://colab.research.google.com/github/Mattlee10/WMSS/blob/main/0420_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

논문 제목: Wearable-based monitoring and self-supervised contrastive learning detect clinical complications during treatment of Hematologic malignancies

- 연구 목적: 웨어러블 데이터를 기반으로 암 치료 중 발생 가능한 합병증(임상 위험)을 조기에 탐지하는 알고리즘 개발

- 데이터 타입: 웨어러블에서 수집된 심박수, 체온, 활동량 등 비정형 센서 데이터

- 기술 접근: Self-supervised contrastive learning(자기 지도 대비 학습) + 시간적 패턴 변화 탐지

- 핵심 결과: 기존 rule-based 방식보다 더 빠르고 정확하게 이상 징후 탐지 가능(합볍증 발생 평균 3.7일 전 조기 탐지)

- 라벨링: 의료 라벨 없이 기계학습만으로 이상 탐지 모델 학습 성공

Step 1. 데이터 로드 & 전처리

무엇을 하는가: 시계열 웨어러블 데이터를 불러와서(또는 시뮬레이션)

- 타임스탬프 기준 일별/시간별로 정렬
- 분당 1샘플 기준으로 리샘플링(resample('1T'))
- 결측치는 보간(interpolate()) 등으로 채움

왜 필요한가:
- 실제 현장 데이터는 누락/불규칙 간격이 잦으므로, 일관된 샘플링이 안 되면 이후 윈도잉에서 창 크기가 달라져 버림
- 깨끗한 연속 데이터를 만들어야 '30분씩 묶었을 때 진짜 평균을 구할 수 있음

In [ ]:
# 1. PhysioNet 데이터셋 접근을 위한 라이브러리 설치
!pip install wfdb --quiet
import wfdb
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 41.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
# 2) drivedb 컬렉션에서 drive01 레코드만 다운로드
wfdb.dl_database(
    'drivedb',
    dl_dir='/content/stress_data',
    records=['drive01']
)

Generating record list for: drive01
Generating list of all files for: drive01
Created local base download directory: /content/stress_data
Finished downloading files


In [ ]:
# 1) 전체 폴더 구조 살펴보기
!ls -R /content/stress_data

print(wfdb.get_record_list('drivedb'))

/content/stress_data:
drive01.dat  drive01.hea
['drive01', 'drive02', 'drive03', 'drive04', 'drive05', 'drive06', 'drive07', 'drive08', 'drive09', 'drive10', 'drive11', 'drive12', 'drive13', 'drive14', 'drive15', 'drive16', 'drive17a', 'drive17b']


In [ ]:
# ───────────────────────────────────────────────
# 1) 레코드 로드
#    - 확장자(.hea/.dat) 없이, 전체 경로를 위치 인자로 전달
# ───────────────────────────────────────────────
record = wfdb.rdrecord('/content/stress_data/drive01')

# ───────────────────────────────────────────────
# 2) DataFrame 변환
# ───────────────────────────────────────────────
df = pd.DataFrame(record.p_signal, columns=record.sig_name)

# ───────────────────────────────────────────────
# 3) 타임스탬프 부여 및 인덱스 설정
# ───────────────────────────────────────────────
df['timestamp'] = pd.date_range(
    start='2025-04-23 00:00:00',
    periods=len(df),
    freq='10L'    # 100Hz → 10ms
)
df.set_index('timestamp', inplace=True)

# ───────────────────────────────────────────────
# 4) 1분 단위 리샘플링 & 결측 보간
# ───────────────────────────────────────────────
df_resampled = (
    df
    .resample('1T')   # 1분 단위
    .mean()
    .interpolate()
)

# ───────────────────────────────────────────────
# 5) 결과 확인
# ───────────────────────────────────────────────
print("Resampled shape:", df_resampled.shape)
df_resampled.head()

Resampled shape: (11, 6)


<ipython-input-4-5d0d31cd7ee3>:15: FutureWarning: 'L' is deprecated and will be removed in a future version, please use 'ms' instead.
  df['timestamp'] = pd.date_range(
<ipython-input-4-5d0d31cd7ee3>:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample('1T')   # 1분 단위


,ECG,EMG,foot GSR,hand GSR,HR,RESP
timestamp,,,,,,
2025-04-23 00:00:00,0.006398,-0.005223,1.982321,10.488052,87.068167,10.828676
2025-04-23 00:01:00,0.004760,-0.007576,1.898895,9.177901,91.831833,11.319675
2025-04-23 00:02:00,-0.000312,-0.005425,3.693971,20.805632,106.473667,11.343718
2025-04-23 00:03:00,0.000868,-0.005115,3.611284,20.729431,99.001500,11.584463
2025-04-23 00:04:00,0.005644,-0.005086,3.464414,20.396722,94.784333,11.721600


Step 2. 윈도잉(Windowing)

무엇을 하는가:
- 30분(=30샘플) 단위로 데이터를 슬라이딩 또는 스템 윈도우로 쪼개서
- 각 윈도우의 평균(또는 중간값, 분산 등)을 추출하여 벡터(임베딩)로 변환

왜 필요한가:
- 1분 단위로 보면 너무 '노이즈'가 많음(심박은 순간변동 큼)
- 30분 평균은 짧은 급변을 걸러 주고, 장기적인 상태 변화에 집중할 수 있게 해 줌
- 백터 하나가 그 구간의 '대표 상태'가 되어, baseline 비교/이상 탐지 시 안정성을 높임

In [ ]:
import pandas as pd
import numpy as np               # ← 여기 추가!
import torch                     # ← PyTorch 사용 시

# 1. 샘플 데이터 생성 (실제 데이터 파일로 교체 가능)
date_rng = pd.date_range(start='2025-04-24 00:00', end='2025-04-24 23:59', freq='T')
df = pd.DataFrame(date_rng, columns=['timestamp'])
df['heart_rate'] = np.random.randint(60, 100, size=len(date_rng))
df['temperature'] = np.random.normal(36.5, 0.5, size=len(date_rng))
df.set_index('timestamp', inplace=True)

# 2. 30분 단위 윈도잉하여 주요 통계량(feature vector) 추출
window_size = '30T'  # 30분
agg_functions = ['mean', 'median', 'std', 'var']

windowed_df = df.resample(window_size).agg(agg_functions)
windowed_df.dropna(inplace=True)  # 결측값 제거

# 3. 임베딩 벡터 추출
embeddings = windowed_df.values  # shape: (num_windows, num_features)

# 4. (선택) PyTorch Tensor로 변환
embeddings_tensor = torch.from_numpy(embeddings).float()

# 결과 확인
print("Windowed Features (first 5 rows):")
print(windowed_df.head())
print("\nEmbeddings Tensor Shape:", embeddings_tensor.shape)

Windowed Features (first 5 rows):
                    heart_rate                               temperature  \
                          mean median        std         var        mean   
timestamp                                                                  
2025-04-24 00:00:00  76.600000   73.0  12.502689  156.317241   36.601419   
2025-04-24 00:30:00  79.366667   80.5  11.084265  122.860920   36.434311   
2025-04-24 01:00:00  72.700000   72.0  10.764405  115.872414   36.579981   
2025-04-24 01:30:00  83.966667   84.5  10.417106  108.516092   36.432686   
2025-04-24 02:00:00  78.933333   78.5  12.289422  151.029885   36.505918   

                                                    
                        median       std       var  
timestamp                                           
2025-04-24 00:00:00  36.589067  0.524468  0.275067  
2025-04-24 00:30:00  36.507470  0.482605  0.232907  
2025-04-24 01:00:00  36.622871  0.573352  0.328732  
2025-04-24 01:30:00  36.337739  0.62805

<ipython-input-6-ba164a905055>:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_rng = pd.date_range(start='2025-04-24 00:00', end='2025-04-24 23:59', freq='T')
<ipython-input-6-ba164a905055>:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  windowed_df = df.resample(window_size).agg(agg_functions)


Step 3. Baseline 임베딩 생성

무엇을 하는가:
- 초기 5일(또는 설정한 정상 기간) 동안의 모든 윈도우 임베딩을 모아서
- 그 평균 벡터(baseline_vector)를 계산
- (보통 정규화 전/후로 나눠서) 전체 임베딩을 StandardScaler로 스케일링

왜 필요한가:
- 개인별 정상 상태를 하나의 참조점으로 정의
- 표준화(StandardScaler)는
  - 서로 다른 센서(심박/온도/가속도) 스케일 차이를 맞춰 주고,
  - 코사인 유사도나 거리 계산을 할 때 왜곡을 방지

- baseline_vector가 이 사람의 정상 평균이 되는 것